# Pivoting a DataFrame Based on Label Values In a Column

The goal of this application is to transform a table (dataframe) with normalized-like structured into a non-normalized structure.

In [7]:
import pandas as pd

### Test DataFrame Creation

In [8]:
def create_dataframe():
    return pd.DataFrame([['A', 1, 2], ['B', 3, 3], ['C', 1, 4]])

In [9]:
create_dataframe()

,0,1,2
0,A,1,2
1,B,3,3
2,C,1,4


### Tranformation Function

In [10]:
def transform(dataframe, label_column_name):

    def _rename_column(dataframe_):
        return dataframe_.rename({0: label_column_name}, axis=1)
    
    def _pivot(dataframe_):
        return dataframe_.pivot(columns=label_column_name)
    
    def _transpose_and_change_values(dataframe_):
        return dataframe_.transpose().fillna(0).astype(int)
    
    def _sort(dataframe_):
        return dataframe_.sort_index(level=1)

    def _reset_index(dataframe_):
        return dataframe_.reset_index().drop('level_0', axis=1)

    def _consolidate(dataframe_):
        dataframe_.iloc[:, 1] = dataframe_.sum(axis=1)
        return dataframe_.drop([1, 2], axis=1)

    df_transformed = _sort(_transpose_and_change_values(_pivot(_rename_column(dataframe))))
    return _consolidate(_reset_index(df_transformed))

In [11]:
transform(create_dataframe(), 'Label') 

,Label,0
0,A,1
1,A,2
2,B,3
3,B,3
4,C,1
5,C,4


### Testing the Performance

In [32]:
%timeit transform(create_dataframe(), 'Label')

8.8 ms ± 86.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### A New Function With Better Performance

In [93]:
def denorm(dataframe, columns):
    
    def _transpose(dataframe_):
        return dataframe_.transpose()
    
    def _rename_columns(dataframe_):
        return dataframe_.rename({ind: val for ind, val in enumerate(df[0])}, axis=1).iloc[1:]
    
    def _stack(dataframe_):
        return dataframe_.stack().sort_index(level=1).reset_index().iloc[:, 1:]

    def _set_new_column_names(dataframe_):
        return dataframe_.rename({col: columns[ind] for ind, col in enumerate(dataframe_.columns)}, axis=1)
    
    return _set_new_column_names(_stack(_rename_columns(_transpose(dataframe))))

In [94]:
denorm(create_dataframe(), ['Label', 'Value'])

,Label,Value
0,A,1
1,A,2
2,B,3
3,B,3
4,C,1
5,C,4


### Final Performance Test

In [96]:
%timeit denorm(create_dataframe(), ['Label', 'Value'])

4.42 ms ± 231 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
